In [ ]:
from google.colab import drive
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Softmax, Embedding, Layer, Attention, TimeDistributed
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! unzip -qq '/content/drive/MyDrive/data/poems/poetry.zip'

In [ ]:
%%time
poem_data = pd.read_csv('robert_frost_collection.csv')
poem_data.dropna(inplace=True)
#poem_data.reset_index(True, inplace=True)

CPU times: user 7.73 ms, sys: 4.34 ms, total: 12.1 ms
Wall time: 20.4 ms


In [ ]:
poem_data.head(10)

,Name,Content,Collection,Year of Publication
1,Stopping by Woods on a Snowy Evening,Whose woods these are I think I know. \nHis ...,New Hampshire,1923.0
2,Fire and Ice,"Some say the world will end in fire,\nSome say...",New Hampshire,1923.0
3,The Aim was Song,Before man came to blow it right\nThe wind onc...,New Hampshire,1923.0
4,The Need of Being Versed in Country Things,The house had gone to bring again\nTo the midn...,New Hampshire,1923.0
5,I Will Sing You One O,It was long I lay\nAwake that night\nWishing t...,New Hampshire,1923.0
6,"For Once, Then, Something",Others taunt me with having knelt at well-curb...,New Hampshire,1923.0
7,The Onset,"Always the same, when on a fated night\nAt las...",New Hampshire,1923.0
8,Nothing Gold Can Stay,"Nature’s first green is gold,\nHer hardest hue...",New Hampshire,1923.0
9,Tree at my Window,"Tree at my window, window tree,\nMy sash is lo...",West-Running Book,1928.0
10,Acquainted with the Night,I have been one acquainted with the night.\nI ...,West-Running Book,1928.0


In [ ]:
data = poem_data['Content']

In [ ]:
corpus_poem = []
text = list(data)
for item in text:
  temp = item.split('\n')
  for i in temp:
    corpus_poem.append(i.lower().strip())

In [ ]:
corpus_sentences = [sentence.strip() + '\n' for sentence in corpus_poem]

In [ ]:
corpus_sentences[:9]

['whose woods these are i think i know.\n',
 'his house is in the village though;\n',
 'he will not see me stopping here\n',
 'to watch his woods fill up with snow.\n',
 'my little horse must think it queer\n',
 'to stop without a farmhouse near\n',
 'between the woods and frozen lake\n',
 'the darkest evening of the year.\n',
 'he gives his harness bells a shake\n']

### Tokenization

In [ ]:
%%time
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus_poem)
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

5604
CPU times: user 74.8 ms, sys: 1.2 ms, total: 76 ms
Wall time: 78 ms


In [ ]:
%%time
sequences = []
for line in corpus_poem:
  token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(token_list)):
    n_gram_seq = token_list[:i+1]
    sequences.append(n_gram_seq)

CPU times: user 64.7 ms, sys: 1.31 ms, total: 66 ms
Wall time: 67.3 ms


In [ ]:
sequences[1:4]

[[286, 257, 234], [286, 257, 234, 54], [286, 257, 234, 54, 4]]

#### Padding

In [ ]:
%%time
max_len = max([len(line) for line in sequences])
sequences = np.array(pad_sequences(sequences, maxlen=max_len, padding='pre'))
sequences[1:4]

CPU times: user 72.9 ms, sys: 2.45 ms, total: 75.4 ms
Wall time: 75.3 ms


array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0, 286, 257, 234],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0, 286, 257, 234,  54],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0, 286, 257, 234,  54,   4]], dtype=int32)

### Feature extraction

In [ ]:
features = sequences[:, :-1]
labels = sequences[:, -1]
labels = to_categorical(labels, vocab_size)

In [ ]:
print(features[2])
print(labels[2])

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 286 257
 234]
[0. 0. 0. ... 0. 0. 0.]


### Model

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 256, input_length=max_len-1))
model.add(LSTM(128, return_sequences=True))
# Attention layer
model.add(Attention())
model.add(Dense(vocab_size, activation='softmax'))
model.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_15 (Embedding)    (None, 19, 256)           1434624   
                                                                 
 lstm_15 (LSTM)              (None, 19, 128)           197120    
                                                                 
 attention_8 (Attention)     (None, 128)               128       
                                                                 
 dense_3 (Dense)             (None, 5604)              722916    
                                                                 
Total params: 2,354,788
Trainable params: 2,354,788
Non-trainable params: 0
_________________________________________________________________


In [ ]:
optimizer = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

history = model.fit(features, labels, epochs=100, verbose=True)

Epoch 1/100
1168/1168 [==============================] - 20s 15ms/step - loss: 6.8731 - accuracy: 0.0508
Epoch 2/100
1168/1168 [==============================] - 8s 7ms/step - loss: 6.4518 - accuracy: 0.0546
Epoch 3/100
1168/1168 [==============================] - 8s 7ms/step - loss: 6.2304 - accuracy: 0.0637
Epoch 4/100
1168/1168 [==============================] - 8s 7ms/step - loss: 5.9487 - accuracy: 0.0771
Epoch 5/100
1168/1168 [==============================] - 8s 7ms/step - loss: 5.6317 - accuracy: 0.0956
Epoch 6/100
1168/1168 [==============================] - 8s 7ms/step - loss: 5.3370 - accuracy: 0.1112
Epoch 7/100
1168/1168 [==============================] - 7s 6ms/step - loss: 5.0538 - accuracy: 0.1298
Epoch 8/100
1168/1168 [==============================] - 8s 7ms/step - loss: 4.7701 - accuracy: 0.1488
Epoch 9/100
1168/1168 [==============================] - 8s 7ms/step - loss: 4.4869 - accuracy: 0.1724
Epoch 10/100
1168/1168 [==============================] - 8s 7ms/step -

### Predictions

In [ ]:
def generate_poem(start_text, words_threshold):
  for j in range(words_threshold):
    token_list = tokenizer.texts_to_sequences([start_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_len-1, padding='pre')
    prediction = model.predict(token_list, verbose=0)
    pred_class = np.argmax(prediction)

    out_text = ""
    for word, indx in tokenizer.word_index.items():
      if indx == pred_class:
        out_text = word
        break
    start_text += " " + out_text
  return start_text

In [ ]:
print(generate_poem('World goes on', 15))

World goes on keeping way to be fault his first ” years burn burn burn hope ’ ’
